<a href="https://colab.research.google.com/github/jagrutimohanty/CMPE-297-Special-Topics-Advance-Deep-Learning/blob/main/Assignment3/Assignment_3_Part_1_297_Sentiment_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install datasets
! pip install transformers

     |████████████████████████████████| 290 kB 4.2 MB/s 
     |████████████████████████████████| 56 kB 5.1 MB/s 
     |████████████████████████████████| 1.3 MB 37.2 MB/s 
     |████████████████████████████████| 243 kB 41.8 MB/s 
     |████████████████████████████████| 125 kB 54.9 MB/s 
     |████████████████████████████████| 160 kB 48.5 MB/s 
     |████████████████████████████████| 271 kB 50.3 MB/s 
     |████████████████████████████████| 3.1 MB 4.1 MB/s 
     |████████████████████████████████| 596 kB 46.8 MB/s 
     |████████████████████████████████| 3.3 MB 41.7 MB/s 
     |████████████████████████████████| 895 kB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


# Loading data - IMDB Dataset for sentiment prediction

In [2]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Downloading:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/84.1M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset imdb downloaded and prepared to /root/.cache/huggingface/datasets/imdb/plain_text/1.0.0/e3c66f1788a67a89c7058d97ff62b6c30531e05b549de56d3ab91891f0561f9a. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# Tokenizing the dataset using bert-base-cased

In [3]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [4]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)


  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

# Sampling data to create train & eval dataset

In [5]:
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [6]:
small_train_dataset

Dataset({
    features: ['attention_mask', 'input_ids', 'label', 'text', 'token_type_ids'],
    num_rows: 1000
})

# Defining Model with Pretrained Model & Classifier

In [7]:
import tensorflow as tf
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=2)

Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
model.summary()

Model: "tf_bert_for_sequence_classification"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  108310272 
_________________________________________________________________
dropout_37 (Dropout)         multiple                  0         
_________________________________________________________________
classifier (Dense)           multiple                  1538      
Total params: 108,311,810
Trainable params: 108,311,810
Non-trainable params: 0
_________________________________________________________________


# Formatting in Tensorflow Format

In [9]:
tf_train_dataset = small_train_dataset.remove_columns(["text"]).with_format("tensorflow")
tf_eval_dataset = small_eval_dataset.remove_columns(["text"]).with_format("tensorflow")

In [10]:
train_features = {x: tf_train_dataset[x] for x in tokenizer.model_input_names}
train_tf_dataset = tf.data.Dataset.from_tensor_slices((train_features, tf_train_dataset["label"]))
train_tf_dataset = train_tf_dataset.shuffle(len(tf_train_dataset)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
eval_tf_dataset = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["label"]))
eval_tf_dataset = eval_tf_dataset.batch(8)

In [11]:
train_features.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

# Training

In [12]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="./logs")
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.metrics.SparseCategoricalAccuracy()
)

model.fit(train_tf_dataset, validation_data=eval_tf_dataset, epochs=3,callbacks=[tensorboard_callback],verbose=1)

Epoch 1/3
125/125 [==============================] - 338s 3s/step - loss: 0.5392 - sparse_categorical_accuracy: 0.7350 - val_loss: 0.3935 - val_sparse_categorical_accuracy: 0.8340
Epoch 2/3
125/125 [==============================] - 310s 2s/step - loss: 0.3011 - sparse_categorical_accuracy: 0.8930 - val_loss: 0.3795 - val_sparse_categorical_accuracy: 0.8500
Epoch 3/3
125/125 [==============================] - 310s 2s/step - loss: 0.1576 - sparse_categorical_accuracy: 0.9440 - val_loss: 0.4281 - val_sparse_categorical_accuracy: 0.8510


In [13]:
from tensorboard import notebook
notebook.display(port=6006, height=1000)

<IPython.core.display.Javascript object>

# Simple component built with gradio

In [ ]:
! pip install gradio

     |████████████████████████████████| 2.0 MB 5.3 MB/s 
     |████████████████████████████████| 1.9 MB 16.4 MB/s 
     |████████████████████████████████| 206 kB 46.6 MB/s 
     |████████████████████████████████| 961 kB 47.7 MB/s 
     |████████████████████████████████| 3.5 MB 48.5 MB/s 
     |████████████████████████████████| 63 kB 2.4 MB/s 
  Created wheel for ffmpy: filename=ffmpy-0.3.0-py3-none-any.whl size=4710 sha256=cc0709d9496fd22291ff7bb5540ac7058832ef046cd641d9b2f5d748c22b2995
  Stored in directory: /root/.cache/pip/wheels/13/e4/6c/e8059816e86796a597c6e6b0d4c880630f51a1fcfa0befd5e6
  Created wheel for flask-cachebuster: filename=Flask_CacheBuster-1.0.0-py3-none-any.whl size=3371 sha256=a20b1981e1431b47d022c2e50729e9dcdf1c9119ec06682c4372ba41427e897c
  Stored in directory: /root/.cache/pip/wheels/28/c0/c4/44687421dab41455be93112bd1b0dee1f3c5a9aa27bee63708
Successfully built ffmpy flask-cachebuster


In [ ]:
import gradio
def get_prediction(text):
  def get_text(prediction):
    if prediction_probs[0][1] > prediction_probs[0][0]:
      return "Positive"
    else:
      return "Negative"
  tokenized_text=tokenizer.encode(text,truncation=True,padding=True,return_tensors="tf")
  prediction=model(tokenized_text)
  prediction_logits = prediction[0]
  prediction_probs = tf.nn.softmax(prediction_logits,axis=1).numpy()
  return get_text(prediction_probs)

iface = gradio.Interface(fn=get_prediction, inputs="text", outputs=["text"])
iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
This share link will expire in 72 hours. If you need a permanent link, visit: https://gradio.app/introducing-hosted
Running on External URL: https://58070.gradio.app


KeyboardInterrupt: ignored